<a href="https://colab.research.google.com/github/andrei-radulescu-banu/stat453-deep-learning-ss21/blob/main/L14/efficientnet-v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install watermark
!rm -rf stat453-deep-learning-ss21
!git clone https://github.com/andrei-radulescu-banu/stat453-deep-learning-ss21.git
import sys, os
sys.path.append("/content/stat453-deep-learning-ss21/L14")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 13.5 MB/s 
Cloning into 'stat453-deep-learning-ss21'...
remote: Enumerating objects: 1275, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 1275 (delta 150), reused 4 (delta 4), pack-reused 1050
Receiving objects: 100% (1275/1275), 116.53 MiB | 30.75 MiB/s, done.
Resolving deltas: 100% (254/254), done.


STAT 453: Deep Learning (Spring 2021)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)  

Course website: http://pages.stat.wisc.edu/~sraschka/teaching/stat453-ss2021/  
GitHub repository: https://github.com/rasbt/stat453-deep-learning-ss21

---

In [2]:
%load_ext watermark
%watermark -a 'Andrei R-B' -v -p torch

Author: Andrei R-B

Python implementation: CPython
Python version       : 3.7.15
IPython version      : 7.9.0

torch: 1.12.1+cu113



# EfficientNet-V2 on Cifar-10

## Imports

In [3]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# From local helper files
from helper_evaluation import set_all_seeds, set_deterministic, compute_confusion_matrix
from helper_train import train_model
from helper_plotting import plot_training_loss, plot_accuracy, show_examples, plot_confusion_matrix
from helper_dataset import get_dataloaders_cifar10, UnNormalize

## Settings and Dataset

In [5]:
##########################
### SETTINGS
##########################

RANDOM_SEED = 123
BATCH_SIZE = 256
NUM_EPOCHS = 50
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
set_all_seeds(RANDOM_SEED)
#set_deterministic()

In [7]:
##########################
### CIFAR-10 DATASET
##########################


train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((120, 120)),
    torchvision.transforms.RandomCrop((110, 110)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      ])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((120, 120)),        
    torchvision.transforms.CenterCrop((110, 110)),            
    torchvision.transforms.ToTensor(),                
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


train_loader, valid_loader, test_loader = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms,
    test_transforms=test_transforms,
    num_workers=2)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Image batch dimensions: torch.Size([256, 3, 110, 110])
Image label dimensions: torch.Size([256])
Class labels of 10 examples: tensor([4, 7, 4, 6, 2, 6, 9, 7, 3, 0])


## Model

In [9]:
import effnetv2
from torchsummary import summary

model = effnetv2.effnetv2_s(num_classes=10)
model = model.to(DEVICE)

summary(model, (3, 110, 110))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 24, 55, 55]             648
       BatchNorm2d-2           [-1, 24, 55, 55]              48
              SiLU-3           [-1, 24, 55, 55]               0
            Conv2d-4           [-1, 24, 55, 55]           5,184
       BatchNorm2d-5           [-1, 24, 55, 55]              48
              SiLU-6           [-1, 24, 55, 55]               0
            Conv2d-7           [-1, 24, 55, 55]             576
       BatchNorm2d-8           [-1, 24, 55, 55]              48
            MBConv-9           [-1, 24, 55, 55]               0
           Conv2d-10           [-1, 24, 55, 55]           5,184
      BatchNorm2d-11           [-1, 24, 55, 55]              48
             SiLU-12           [-1, 24, 55, 55]               0
           Conv2d-13           [-1, 24, 55, 55]             576
      BatchNorm2d-14           [-1, 24,

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list, train_acc_list, valid_acc_list = train_model(
    model=model,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader,
    valid_loader=valid_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

plot_training_loss(minibatch_loss_list=minibatch_loss_list,
                   num_epochs=NUM_EPOCHS,
                   iter_per_epoch=len(train_loader),
                   results_dir=None,
                   averaging_iterations=200)
plt.show()

plot_accuracy(train_acc_list=train_acc_list,
              valid_acc_list=valid_acc_list,
              results_dir=None)
plt.ylim([60, 100])
plt.show()

Epoch: 001/050 | Batch 0000/0175 | Loss: 2.3028
Epoch: 001/050 | Batch 0100/0175 | Loss: 1.5503
Epoch: 001/050 | Train: 47.92% | Validation: 47.32%
Time elapsed: 3.33 min
Epoch: 002/050 | Batch 0000/0175 | Loss: 1.4057
Epoch: 002/050 | Batch 0100/0175 | Loss: 1.0613
Epoch: 002/050 | Train: 61.43% | Validation: 60.48%
Time elapsed: 6.69 min
Epoch: 003/050 | Batch 0000/0175 | Loss: 1.0143


In [ ]:
model.cpu()
unnormalizer = UnNormalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
class_dict = {0: 'airplane',
              1: 'automobile',
              2: 'bird',
              3: 'cat',
              4: 'deer',
              5: 'dog',
              6: 'frog',
              7: 'horse',
              8: 'ship',
              9: 'truck'}

show_examples(model=model, data_loader=test_loader, unnormalizer=unnormalizer, class_dict=class_dict)

In [ ]:
mat = compute_confusion_matrix(model=model, data_loader=test_loader, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())
plt.show()